In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date
import plotly.plotly as py
import plotly.graph_objs as go
from itertools import product
import plotly.figure_factory as ff
import ast

In [2]:
edges = pd.read_csv('checkedge.csv')

In [3]:
edges = edges.rename(columns = {'Jul-15 to Aug08': 'CurrMonth'})
col = 'CurrMonth'

In [4]:
series = pd.Series([ast.literal_eval(i) for i in edges['areabrkup']])
srdf= pd.DataFrame(series, columns = ['AreaSplit'])
edges = edges.merge(srdf, left_index=True, right_index=True)

In [5]:
def printheatmap (areadict):
    heatmapdf = pd.DataFrame([[None,None,None]], columns=['OrgArea','DestArea','Wt'])
    for count, keys in enumerate(list(areadict.keys())):
        datalist = []
        hmdict = areadict.get(keys)[0]
        hmdf = pd.DataFrame.from_dict(hmdict, orient='index', dtype=None)
        hmdf = hmdf.reset_index()
        hmdf.columns = ['Area','Wt']
        hmdf['OrgArea'] = hmdf.apply(lambda x: x['Area'].split('-')[0],axis=1)
        hmdf['DestArea'] = hmdf.apply(lambda x: x['Area'].split('-')[1],axis=1)
        hmdf = hmdf[heatmapdf.columns]
        heatmapdf = heatmapdf.append(hmdf)
    heatmapdf = heatmapdf.dropna()
    heatmappivot = heatmapdf.pivot_table(index = 'OrgArea', values = 'Wt', columns = 'DestArea', aggfunc = pd.np.sum)
    heatmappivot = heatmappivot.fillna(0.0).T
    x = list(heatmappivot.columns)
    y = list(heatmappivot.index)
    z = heatmappivot.values
    z_text = np.around(z,decimals=1)
    xaxis = {'dtick': 1,'gridcolor': 'rgb(0, 0, 0)','side': 'bottom','ticks': ''}
    heatmapfig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
    for i in range(len(heatmapfig.layout.annotations)):
        heatmapfig.layout.annotations[i].font.size = 8
    heatmapfig.layout.title = 'Load Table in Mt - XAxis OrgArea YAxis DestArea'
    heatmapfig.layout.xaxis = xaxis
    return heatmapfig



In [6]:
def printfigs (orglist,destlist):    
    reporttitle = 'Linehaul Edge Data'
    runlist = [i for i in product(orglist,destlist)]
    datalist = []
    areadict = {}
    for i in runlist:
        org = i[0]
        dest = i[1]
        try:
            fl= edges[(edges['Origin']==org)&(edges['Destn']==dest)]
            bl= edges[(edges['Origin']==dest)&(edges['Destn']==org)]
            for df in [fl,bl]:
                org = (list(df['Origin'])[0])
                dest = (list(df['Destn'])[0])
                ydata = (list(df[col])[0])
                var = (list(df['Var'])[0])
                areasplit = list(df['AreaSplit'].values)
                areadict.update({(org,dest): areasplit})
                trace = go.Bar(
                x = '{0}-{1}'.format(org,dest),
                y = ydata,
                text = 'Edge: {0}, Var: {1}'.format(ydata,var),
                textposition='bottom',
                name = '{0}-{1}'.format(org,dest)
                )
                datalist.append(trace)
        except:
            pass
    layout = go.Layout(
        title=reporttitle,
        barmode='stack'
    )
    fig = go.Figure(data=datalist, layout=layout)
    heatmapfig = printheatmap(areadict)
    return fig,heatmapfig
        
    

In [7]:
fig,heatmapfig = printfigs(['BOMH'],['AMCH','DELH','IDRH'])
py.iplot(fig, filename='stacked-bar')

In [8]:
py.iplot(heatmapfig, filename='heatmap')